In [ ]:
import threading
import queue
import time

##### Declaring the number of nodes

In [ ]:
n = 12

##### Declaring Node class

In [ ]:
class Node:
    def __init__(self, node_id, total_nodes, queues):
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.queues = queues
        self.in_queue = queues[node_id]
        self.prev_queue = queues[node_id - 1]
        self.next_queue = queues[(node_id + 1) % total_nodes]
        self.status = 'unknown'

    def send_prev(self, data):
        self.prev_queue.put(data)

    def send_next(self, data):
        self.next_queue.put(data)

    def accept(self):
        data = self.in_queue.get()
        self.in_queue.task_done()
        return data

#### LCR algorithm

In [ ]:
def lcr_node_worker(node):
    u = node.node_id
    send = node.node_id

    while True:
        node.send_next(send)

        v = node.accept()
        if v == "TERM":
            break

        if v is None:
            send = v
            continue
        if v > u:
            send = v
            node.status = 'non-leader'
        elif v < u:
            send = None
        else:
            node.status = 'leader'
            return

In [ ]:
def lcr_run(total_nodes):
    queues = [queue.Queue() for _ in range(total_nodes)]
    nodes = [Node(i, total_nodes, queues) for i in range(total_nodes)]

    threads = []
    for node in nodes:
        t = threading.Thread(target=lcr_node_worker, args=(node,))
        t.start()
        threads.append(t)

    time.sleep(3)
    for q in queues:
        q.put("TERM")

    for t in threads:
        t.join()

    for node in nodes:
        print(f"{node.node_id}: {node.status}")
    print("\nLCR algorithm complete.")

In [ ]:
lcr_run(n)

#### HS algorithm

In [ ]:
def hs_node_worker(node):
    u = node.node_id
    send_plus = ('from_prev', u, 'out', 1)
    send_minus = ('from_next', u, 'out', 1)
    phase = 0

    while True:
        node.send_next(send_plus)
        node.send_next(send_minus)

        v_prev, v_next = None, None
        while v_prev is None or v_next is None:
            v = node.accept()
            if v == "TERM":
                return
            if v is None:
                continue
            if v[0] == 'from_prev':
                v_prev = v[1:]
            if v[0] == 'from_next':
                v_next = v[1:]

        if v_prev[1] == 'out':
            v, out, h = v_prev
            if v > u and h > 1:
                send_plus = ('from_prev', v, 'out', h - 1)
            elif v > u and h == 1:
                send_minus = ('from_next', v, 'in', 1)
            elif v == u:
                node.status = 'leader'

        if v_next[1] == 'out':
            v, out, h = v_next
            if v > u and h > 1:
                send_minus = ('from_next', v, 'out', h - 1)
            elif v > u and h == 1:
                send_plus = ('from_prev', v, 'in', 1)
            elif v == u:
                node.status = 'leader'

        if v_prev[1] == 'in' and v_prev[2] == 1 and v_prev[0] != u:
            v, in_, h = v_prev
            send_plus = ('from_prev', v, 'in', 1)
        if v_next[1] == 'in' and v_next[2] == 1 and v_next[0] != u:
            v, in_, h = v_next
            send_minus = ('from_next', v, 'in', 1)
        if (v_prev[0] == u and v_prev[1] == 'in' and v_prev[2] == 1) and (v_next[0] == u and v_next[1] == 'in' and v_next[2] == 1):
            phase += 1
            send_plus = ('from_prev', u, 'out', 2 ** phase)
            send_minus = ('from_next', u, 'out', 2 ** phase)


In [ ]:
def hs_run(total_nodes):
    queues = [queue.Queue() for _ in range(total_nodes)]
    nodes = [Node(i, total_nodes, queues) for i in range(total_nodes)]

    threads = []
    for node in nodes:
        t = threading.Thread(target=hs_node_worker, args=(node,))
        t.start()
        threads.append(t)

    time.sleep(3)
    for q in queues:
        q.put("TERM")

    for t in threads:
        t.join()

    for node in nodes:
        print(f"{node.node_id}: {node.status}")
    print("\nHS algorithm complete.")

In [ ]:
hs_run(n)